In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os

In [20]:
df = pd.read_csv("./data/thinktank_citations_tdm.csv", index_col=0)

In [23]:
df['sentence'][0].strip()

'"</p><p>Others analysts were more critical, saying the Saudis had exploited Mr. Trump by seizing on the good will generated during his visit to carry out a long-planned move against a smaller neighbor.</p><p>"The Saudis played Donald Trump like a fiddle," said Bruce O. Riedel, a former intelligence analyst who advised Mr. Obama and now works at the Brookings Institution.'

In [25]:
comp = pd.read_csv("./data/components_data/thinktank_citations.csv")